# Homework 4 (Ibotta Products)

In [1]:
# import analysis packages
import keras
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

In [49]:
# read data from .csv files
trainDF = pd.read_csv('./ibotta_train.csv')
testDF = pd.read_csv('./ibotta_test.csv')

# drop category code field
trainDF.drop('Cat_code', axis = 1, inplace = True)

# combine data sets for preprocessing
trainDF['origin'] = 'train'
testDF['origin'] = 'test'
fullDF = pd.concat([trainDF, testDF])

# combine brand and name fields
fullDF['Brand_name'].where(-fullDF['Brand_name'].isna(), '', inplace = True)
fullDF['full_text'] = fullDF['Brand_name'] + ' ' + fullDF['Name']

# check dimensions of data
display(trainDF.shape)
display(testDF.shape)
display(fullDF.shape)

# check data types
display(fullDF.info())

# peek at data
display(fullDF.head())

(8000, 5)

(1999, 4)

(9999, 6)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9999 entries, 0 to 1998
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Id          9999 non-null   int64 
 1   Name        9999 non-null   object
 2   Brand_name  9999 non-null   object
 3   Category    8000 non-null   object
 4   origin      9999 non-null   object
 5   full_text   9999 non-null   object
dtypes: int64(1), object(5)
memory usage: 546.8+ KB


None

,Id,Name,Brand_name,Category,origin,full_text
0,1,"4C HomeStyle Parmesan Grated Cheese, 6 oz",4C,Dairy,train,"4C 4C HomeStyle Parmesan Grated Cheese, 6 oz"
1,2,"4C 100% Natural Parmesan Cheese, 6 oz",,Dairy,train,"4C 100% Natural Parmesan Cheese, 6 oz"
2,3,"4C Parmesan & Romano Cheese, Grated",4C,Dairy,train,"4C 4C Parmesan & Romano Cheese, Grated"
3,4,Advance Fast Fixin' Country Fried Steaks with ...,Fast Fixin',Frozen Foods,train,Fast Fixin' Advance Fast Fixin' Country Fried ...
4,5,Borden: Salted 4 Ct Butter,Borden Dairy Company,Dairy,train,Borden Dairy Company Borden: Salted 4 Ct Butter


In [57]:
np.mean(fullDF.loc[fullDF['Brand_name'] != ''].apply(lambda x: x['Name'].find(x['Brand_name']), axis = 1) != -1)

0.5648956817079088